#1. Segundo projeto do curso de deep learning usando pytorch

##2. Importando as bibiliotecas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import MaxAbsScaler
import torch 
import torch.nn as nn

###2.1 Instalar a biblioteca skorch

In [ ]:
!pip install skorch==0.8.0

     |████████████████████████████████| 113 kB 5.2 MB/s 


In [ ]:
import skorch

##3. Deixando o fator randomico de reprodutibilidade padrão em 1

In [ ]:
np.random.seed(1)
torch.manual_seed(1)

##4. Lendo os dados dos arquivos csv e plotando a classe

In [ ]:
previsores = pd.read_csv('/content/drive/MyDrive/entradas_breast.csv')
predito = pd.read_csv('/content/drive/MyDrive/saidas_breast.csv')
print(predito.shape)
quantidade_unico = np.unique(predito)
print('Classes possíveis: {}'.format(quantidade_unico))

(569, 1)
Classes possíveis: [0 1]


##5. Redefinindo as variaveis para np array

In [ ]:
previsoes = np.array(previsores,dtype='float32')
classes = np.array(predito,dtype='float32').squeeze(1)#tira a dimensão extra do array

##6. Normalizar as entradas

In [ ]:
previsoes_norm = MaxAbsScaler().fit(previsoes)
previsoes_norm = previsoes_norm.transform(previsoes)

##7. Construir a classe para validação cruzada

In [ ]:
class classificador(nn.Module):
#na validação cruzada, temos que determindar explicitamente a inicialização dos pesos
#a validação dos pesos é feita em torch.nn.init.uniform(self.camadax.weight)
#aqui tambem vamos colocar uma camada de droupout
  def __init__(self):#inicia o construtor
    super().__init__()#inicializa o construtor da super classe
    self.camada0 = nn.Linear(in_features = 30, out_features = 16)
    torch.nn.init.uniform_(self.camada0.weight)
    self.ativacao0 = nn.ReLU()
    self.dp0 = nn.Dropout(0.2)
    self.camada1 = nn.Linear(in_features = 16, out_features = 16)
    torch.nn.init.uniform_(self.camada1.weight)
    self.ativacao1 = nn.ReLU()
    self.dp1 = nn.Dropout(0.2)
    self.camada2 = nn.Linear(in_features=16, out_features=1)
    torch.nn.init.uniform_(self.camada2.weight)
    self.output = nn.Sigmoid()
  def forward(self, x):
    x = self.camada0(x)
    x = self.ativacao0(x)
    x = self.dp0(x)
    x = self.camada1(x)
    x = self.ativacao1(x)
    x = self.dp1(x)
    x = self.camada2(x)
    x = self.output(x)
    return x

##8. Criar a validação cruzada com o skorch

In [ ]:
classificador_sklearn = skorch.NeuralNetBinaryClassifier(module=classificador,
                                                         criterion = torch.nn.BCELoss,
                                                         optimizer = torch.optim.Adam,
                                                         lr=0.001, 
                                                         optimizer__weight_decay=0.0001,
                                                         max_epochs = 100,
                                                         batch_size = 10,
                                                         train_split = False
                                                         )

##9. Criar a cross validation com sklearn

In [ ]:
treinamento = cross_val_score(classificador_sklearn,previsoes_norm,classes,cv=5,scoring='accuracy')

##10. Verificando métricas de resultado

In [ ]:
#Vendo a média dos resultados
media_result = treinamento.mean()
#Vendo o desvio padrão dos resultados
desvio_result = treinamento.std()
print('Média: {}\nDesvio: {}'.format(media_result,desvio_result))

Média: 0.9192206179164726
Desvio: 0.028405314268504105
